# Tutorial on cross-matching Gaia DR3 and the NASA exoplanet archive

## Author: Anthony Brown (Leiden Observatory)

## __Before you start (if not already done)__
A number of the exercises below require you to be registered as a user of the Gaia archive. If you plan on requesting data from the Gaia archive regularly this is in any case a good idea. The registration easy and quick (and free) and offers the following facilities:

* 20 GB of space to store query results
* the option to create and share your own tables ('user tables'),
* longer timeout for large or complex query jobs.

You can register [here](https://www.cosmos.esa.int/web/gaia-users/register).

## First install required Python packages

We need the `astroquery` package to access the tools for programmatic access to the Gaia archive. For more information and examples see:
* [astroquery documentation](https://astroquery.readthedocs.io/en/latest/gaia/gaia.html)
* [Gaia archive python tutorial](https://www.cosmos.esa.int/web/gaia-users/archive/use-cases#ClusterAnalysisPythonTutorial)


## Python imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc

from astroquery.gaia import Gaia
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive

from astropy.table import Table

from itertools import chain

## Matplotlib settings

In [ ]:
linewidths = 2
axislinewidths = 2
lenticks = 6

rc('font', family='sans-serif', size=18)
rc('xtick.major', size=lenticks)
rc('xtick.minor', size=lenticks * 2 / 3)
rc('ytick.major', size=lenticks)
rc('ytick.minor', size=lenticks * 2 / 3)
rc('lines', linewidth=linewidths)
rc('axes', linewidth=axislinewidths)

## Find Gaia DR3 tables in the Gaia archive
We start by generating an overview of the Gaia DR3 tables available from the [ESA Gaia archive](https://gea.esac.esa.int/archive/).

In [ ]:
tables = Gaia.load_tables(only_names=True)
for table in (tables):
    if 'gaiadr3' in table.get_qualified_name():
        print (table.get_qualified_name())

## A first query on the Gaia Catalogue of Nearby Stars (GCNS)

Request data for all GCNS sources within 20 pc and store the results in `gtable`.

__Note:__ The GCNS table lives in `external.gaiaedr3_gcns_main_1` in the Gaia archive.

In [ ]:
job = Gaia.launch_job_async("SELECT source_id, ra, dec, parallax, phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag \
FROM  external.gaiaedr3_gcns_main_1 \
WHERE parallax>50")

gtable = job.get_results()

In [ ]:
print(gtable.info)

We see that the query results in 2575 sources of which a few do not have G-band photometry or (BP-RP) colours available. The next code block makes a colour magnitude diagram from the data in the table.

In [ ]:
bp_rp = gtable['phot_bp_mean_mag']-gtable['phot_rp_mean_mag']

fig, axcmd = plt.subplots(1, 1, figsize=(7, 7))

axcmd.hexbin(bp_rp, gtable['phot_g_mean_mag'], bins='log', mincnt=1)
axcmd.set_xlabel(r'$(G_\mathrm{BP}-G_\mathrm{RP})$')
axcmd.set_ylabel(r'$G$')
axcmd.invert_yaxis()

plt.show()

## __QUESTION 1__
Plot the observational Hertzsprung-Russell diagram for this sample of stars. You can modify the above example to plot absolute magnitude on the y-axis.

In [ ]:
# Your plotting code here

## __QUESTION 2__
Modify the above query so you can plot the observational HR diagram in the SDSS filter system (for example using the g and i bands).

Hint: Look up in the [astroquery.gaia documentation](https://astroquery.readthedocs.io/en/latest/gaia/gaia.html) how you can inspect the contents of a specific table. You can find column names and descriptions. Alternatively, you can inspect the contents of `external.gaiaedr3_gcns_main_1
` to find the relevant column names.

In [ ]:
# Code to inspect the table here

In [ ]:
# Code to select the SDSS photometry from the GCNS table.

In [ ]:
# Code to plot the SDSS photometry version of the observational HRD

## Query the NASA Exoplanet Archive
We are interested in combining the Gaia DR3 data with the data from the [NASA Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/) in order to use the astrophysical properties the stars in Gaia DR3 to improve the characterisation of exoplanets.

We first inspect the Exoplanet Archive contents using the `astroquery` [tools for the Exoplanet Archive](https://astroquery.readthedocs.io/en/latest/ipac/nexsci/nasa_exoplanet_archive.html). We query the 'Planetary systems composite data' table. The various columns asked for in the query are needed in the following exercises.

In [ ]:
exocolumns = ['pl_name', 'hostname', 'ra', 'dec', 'sy_gaiamag', 'st_teff', 'st_logg', 'st_met', 'st_lum', 'st_rad', 'st_age']
select_string = ", ".join(exocolumns)
exotable = Table(NasaExoplanetArchive.query_criteria(table="pscomppars", select=select_string))

In [ ]:
exotable.pprint()

## Cross-matching the Exoplanet Archive to Gaia DR3
We will use a positional cross-match on the source coordinates to do the cross-match between the exoplanet host stars in the Exoplanet Archive and Gaia DR3.

### __Login to the Gaia archive first__

For this exercise you must be logged in to the Gaia archive. The form in the next code cell asks for you archive user name. This is also needed further down.

*Your might be automatically logged out after a while. Just re-run this cell to login again.*

In [ ]:
#Provide your Gaia archive user name - change 'your_username' to your Gaia login
username = 'your_username'

Gaia.login(user = username)


### Cleaning up to avoid archive errors

*You can skip this step if you have yet not created any tables.*

As part of the exercise you will be uploading and generating user tables in the Gaia archive. To avoid errors due to tables already existing (in case you need to restart the exercise) you can execute the following code block
which removes tables that are generated during one of the exercises below. If the tables were already deleted you will get an error message.



In [ ]:
Gaia.delete_user_table('exoplanet_archive')
Gaia.delete_user_table('exoplanets_gaia')
Gaia.delete_user_table('exo_gaia_best')

You can also delete the tables through the Gaia archive [web interface](https://gea.esac.esa.int/archive/) (after signing in). To delete a table in the archive, look the table up in your "User Tables" section in the menu to the left of the screen (see image below) and then check the table(s) to delete and click on the "delete" icon
(highlighted in the image).

<img src="https://catcopy.ipac.caltech.edu/ssw/hands-on/Gaia_login.png"></img>

### Cross-match steps

### Step 1
We upload the table obtained from the Exoplanet Archive to the Gaia archive. When uploading we omit the 'sky_coord' column as this cannot be ingested automatically and is not needed.

In [ ]:
Gaia.upload_table(upload_resource=exotable[exocolumns], table_name='exoplanet_archive')

### Step 2
We do a positional cross-match with the query in the code below. We use a search radius of 1 arcsec to look for Gaia sources matching the planet host star.

__NOTE 1__ The full name of the table we just uploaded will be `user_<your_username>.exoplanet_archive`. The first line of code takes care of this.

__NOTE 2__ We have to mark the 'ra' and 'dec' columns in the Gaia archive as the columns containing the sky coordinates, otherwise the cross-match will not work.

In [ ]:
# Construct the correct table name
full_qualified_table_name = 'user_' + username + '.exoplanet_archive'
xmatch_table_name = 'exoplanets_gaia'
search_radius = 1.0

# Mark the ra and dec columns as containing the sky coordinates
Gaia.update_user_table(table_name=full_qualified_table_name, 
                       list_of_changes=[["ra","flags","Ra"], ["dec","flags","Dec"]])

# Run the cross-match between our user table and the gaiadr3.gaia_source table
Gaia.cross_match(full_qualified_table_name_a=full_qualified_table_name,
                 full_qualified_table_name_b='gaiadr3.gaia_source',
                 results_table_name=xmatch_table_name, radius=search_radius)

### Step 3
The query above generated a table in the archive called `user_<your_username>.exoplanets_gaia`. It contains only three columns
* `exoplanet_archive_exoplanet_archive_oid`
* `gaia_source_source_id`
* `separation`

We can use this data to construct the table with the Gaia DR3 data we would like to have for the exoplanet host stars. The next code block creates the query which requests a subset of the `gaiadr3.gaia_source` data. We will get to retrieving the Gaia DR3 astrophysical parameters for the exoplanets hosts in a subsequent exercise.

*You can ignore the warnings about "Unit not supported".*

In [ ]:
xmatch_table = 'user_' + username + '.' + xmatch_table_name
gaia_columns = ['source_id', 'ra', 'dec', 'parallax', 'parallax_over_Error', 'phot_g_mean_mag', 
                'phot_bp_mean_mag', 'phot_rp_mean_mag', 'ag_gspphot', 'ebpminrp_gspphot', 'distance_gspphot']
gaia_columns_for_query = []
for name in gaia_columns:
    gaia_columns_for_query.append('gaia.'+name)
gaia_select = ", ".join(gaia_columns_for_query)

query = 'SELECT xm."separation"*3600 as sep, ' + gaia_select + ', exopl.* FROM gaiadr3.gaia_source AS gaia, ' + \
full_qualified_table_name + ' AS exopl, ' + xmatch_table + ' AS xm ' + \
'WHERE (xm.gaia_source_source_id = gaia.source_id AND exoplanet_archive_exoplanet_archive_oid = exopl.exoplanet_archive_oid)'

job = Gaia.launch_job_async(query=query, name='exoplanet_archive_dr3')

exo_gaia_table = job.get_results()

In [ ]:
print(f"Number of matches found for search radius {search_radius:.1f} arcsec: {exo_gaia_table['sep'].size}")

## __QUESTION 3__
We now have a table with all the Gaia DR3 data for the host stars from the Exoplanet Archive. Not all host stars have a match in Gaia DR3 and how do we known the matches found are correct? Are all host stars uniquely matched to a Gaia source?

__3.1__ Come up with a basic check of the cross-matches. Think about plots you could make to spot any matches that might be dubious.

__3.2__ Identify exoplanets matched to more than one Gaia DR3 source.

__3.3__ Use the result from 3.2 to decide on a criterion for identifying bad matches. Find a way to filter these out of the created table and then make a table with only the best matches called `exo_gaia_best`.

__HINT__ For question 3.1 consider what criterion other than the proximity of two sources would convince you that the match is correct. 

__HINT__ For questions 3.2 and 3.3 Python's [list facilities](https://docs.python.org/3/tutorial/introduction.html#lists), see also the tutorial on [data structures](https://docs.python.org/3/tutorial/datastructures.html), and the [`itertools`](https://docs.python.org/3/library/itertools.html?highlight=itertools) package are useful. See if you can find a way to create a list of exoplanet names which occur more than once, and then find a way to list the `exo_gaia_table` rows that correspond to these multiple entries.

In [ ]:
# Code for question 3.1

In [ ]:
# Code for question 3.2
# This is somewhat tricky so don't hesitate to ask for more hints!

In [ ]:
# Code for question 3.3

## __QUESTION 4__
Plot the Gaia observational HR diagram for the exoplanet hosts stars, corrected for the effects of extinction.

__4.1__ Make two plots, one using the parallaxes to calculate $M_{G,0}$ and the other using Gaia DR3 distances.

__4.2__ What might be the cause of the differences you see? Create a plot to investigate this.

*NOTE: not all stars have a distance estimate in Gaia DR3.*

In [ ]:
# Code for question 4.1

In [ ]:
# Code for question 4.2

## Extracting astrophysical parameters from Gaia DR3
The main `gaiadr3.gaia_source` table in the archive does contain astrophysical parameters but here we will use the `gaiadr3.astrophysical_parameters` table to extract more information than available from `gaia_source`. We focus here on the parameters determined with the 'GSP-Phot' module of the Gaia/DPAC processing pipelines. These are listed as `*_gspphot`.

This is done by creating a user table from our exoplanet-Gaia table using only the best matches with a distance listed in Gaia DR3 (if there is no distance listed there are also no`*_gspphot` astrophysical parameters available).

*You can ignore the warnings about Invalid unit strings.*

__NOTE__ In the code below assumes you have already created the `exo_gaia_best` table.

In [ ]:
exo_gaia_best_dist = exo_gaia_best[exo_gaia_best['distance_gspphot']>0]

Gaia.upload_table(upload_resource=exo_gaia_best_dist, table_name='exo_gaia_best')

Now that the table is uploaded to the archive we can run a query to extract the astrophysical parameters of the host stars. This is done through a 'join' between our user table and the `astrophysical_parameters` table.

In [ ]:
exo_table_name = 'user_' + username + '.exo_gaia_best'
ap_columns = ['teff_gspphot', 'logg_gspphot', 'mh_gspphot', 'radius_flame', 'lum_flame', 'mass_flame', 'age_flame']
ap_columns_for_query = []
for name in ap_columns:
    ap_columns_for_query.append('aps.'+name)
ap_select = ", ".join(ap_columns_for_query)

query = 'select exo.*, ' + ap_select + ' from ' + exo_table_name + \
' as exo join gaiadr3.astrophysical_parameters as aps using (source_id)'

job = Gaia.launch_job_async(query=query, name='exoplanets_dr3_aps')

exo_dr3_aps = job.get_results()

In [ ]:
print(exo_dr3_aps.info)

The resulting table contains the data for 4292 exoplanet host stars, where not all Gaia DR3 astrophysical parameters are available for all stars.

## __QUESTION 5__
Gaia DR3 also contains a table with a sample of FGKM stars with very well determined astrophysical parameters. The sample is described in the paper [Gaia Data Release 3: A Golden Sample of Astrophysical Parameters by Gaia Collboration, Creevey et al. (2022)](https://arxiv.org/pdf/2206.05870.pdf).

Modify the above query to extract the astrophysical parameters for the exoplanet host stars in the FGKM golden sample (selecting all columns in the table is fine).

__HINT__ The table with the FGKM golden sample can be found in the `performance verification` section of the Gaia data release 3 tables (see archive web interface) and the table already contains the relevant astrophysical parameters.

In [ ]:
# Code for question 5

## __Challenge for the rest of the week__

You now have all the data in hand to do the following as an extra challenge during the remainder of the workshop.

1. How do the exoplanet host star astrophysical parameters from the NASA Exoplanet Archive and Gaia DR3 compare?
2. Can you recalculate the planetary radii using the new Gaia DR3 astrophysical data?
3. What changes can you see in the planet population properties when using the Gaia DR3 data?
4. We did not match all 5044 planets in the table created from the Exoplanet Archive. Perhaps our search radius was to strict. You can repeat exercise 3 with a larger search radius to see if you can recover more matches of planet host stars to Gaia DR3.